In [1]:
import pandas as pd
import numpy as np

## Making sequence count data sets from pruned GISAID metadata

The `giasaid_metadata_pruned.tsv` is generated from the Nextstrain-curated GISAID metadata. Details for this can be found in `../data/README.md`.

In [2]:
# https://data.nextstrain.org/files/ncov/open/metadata.tsv.gz
raw_metadata_reader = pd.read_csv(
    "../data/gisaid_metadata_pruned.tsv", 
    sep="\t",
    usecols=(
        "strain",
        "date",
        "date_submitted",
        "country",
        "division",
        "Nextstrain_clade"
        ),
    dtype={
        "country": "category",
        "division": "category",
        "Nextstrain_clade": "category",
        },
    chunksize=100000)

In [3]:
# Countries of interest
locations = ["USA", "Japan", "United Kingdom", "Australia", "Brazil", "South Africa"]
start_date = "2022-01-01"
end_date = "2022-06-30"

# Borrowing from John here
def clean_metadata(metadata):
    # Filter to countries
    metadata = metadata[metadata["country"].isin(locations)]
    
    # Remove ambigious dates
    unambiguous_dates = (metadata["date"] != "?") & (metadata["date"].str.count("-") == 2)
    unambiguous_dates = unambiguous_dates & ~(metadata["date"].str.contains("X", na=True))
    metadata = metadata[unambiguous_dates].copy()
    
    # Filter to date range
    metadata["date"] = pd.to_datetime(metadata["date"])
    metadata["date_submitted"] = pd.to_datetime(metadata["date_submitted"])

    date_since_start_date = (metadata["date"] >= start_date)
    date_before_end_date = (metadata["date"] <= end_date)
    metadata = metadata[(date_since_start_date) & (date_before_end_date)].copy()
        
    # Filter records without NS clade
    metadata = metadata[~pd.isnull(metadata["Nextstrain_clade"])].copy()
    
    # Remove bad sequences (https://github.com/blab/rt-from-frequency-dynamics/tree/master/data/omicron-countries-split)
    return metadata

In [4]:
metadata_chunks = [clean_metadata(chunk) for chunk in raw_metadata_reader]

In [5]:
metadata = pd.concat(metadata_chunks, ignore_index=True)
metadata

,strain,date,date_submitted,country,division,Nextstrain_clade
0,AmericanSamoa/AS-H2211054/2022,2022-04-15,2022-05-28,USA,American Samoa,21L (Omicron)
1,AmericanSamoa/AS-H2211055/2022,2022-04-15,2022-05-28,USA,American Samoa,21L (Omicron)
2,AmericanSamoa/AS-H2211056/2022,2022-04-19,2022-05-28,USA,American Samoa,21L (Omicron)
3,AmericanSamoa/AS-H2211057/2022,2022-04-19,2022-05-28,USA,American Samoa,21L (Omicron)
4,AmericanSamoa/AS-H2211058/2022,2022-04-23,2022-05-28,USA,American Samoa,21K (Omicron)
...,...,...,...,...,...,...
2535277,env/USA/CA-SEARCH-91226/2022,2022-05-18,2022-07-22,USA,California,22C (Omicron)
2535278,env/USA/WI-MHDL-2022050002/2022,2022-04-26,2022-05-16,USA,Wisconsin,21L (Omicron)
2535279,lion/USA/TX-22-002383-002/2022,2022-01-14,2022-03-04,USA,Texas,21J (Delta)
2535280,tiger/USA/FL-22-000348-001/2022,2022-01-02,2022-03-04,USA,Florida,21J (Delta)


In [6]:
# Map clade to variant
clade_to_variant = {"21A (Delta)" : "Delta", 
                    "21I (Delta)" : "Delta", 
                    "21J (Delta)" : "Delta", 
                    "21K (Omicron)" : "Omicron 21K", 
                    "21L (Omicron)" : "Omicron 21L", 
                    "22A (Omicron)" : "Omicron 22A", 
                    "22B (Omicron)" : "Omicron 22B", 
                    "22C (Omicron)" : "Omicron 22C"}

metadata["variant"] = (metadata["Nextstrain_clade"]
                       .map(clade_to_variant)
                       .fillna("other")
                      )
metadata

,strain,date,date_submitted,country,division,Nextstrain_clade,variant
0,AmericanSamoa/AS-H2211054/2022,2022-04-15,2022-05-28,USA,American Samoa,21L (Omicron),Omicron 21L
1,AmericanSamoa/AS-H2211055/2022,2022-04-15,2022-05-28,USA,American Samoa,21L (Omicron),Omicron 21L
2,AmericanSamoa/AS-H2211056/2022,2022-04-19,2022-05-28,USA,American Samoa,21L (Omicron),Omicron 21L
3,AmericanSamoa/AS-H2211057/2022,2022-04-19,2022-05-28,USA,American Samoa,21L (Omicron),Omicron 21L
4,AmericanSamoa/AS-H2211058/2022,2022-04-23,2022-05-28,USA,American Samoa,21K (Omicron),Omicron 21K
...,...,...,...,...,...,...,...
2535277,env/USA/CA-SEARCH-91226/2022,2022-05-18,2022-07-22,USA,California,22C (Omicron),Omicron 22C
2535278,env/USA/WI-MHDL-2022050002/2022,2022-04-26,2022-05-16,USA,Wisconsin,21L (Omicron),Omicron 21L
2535279,lion/USA/TX-22-002383-002/2022,2022-01-14,2022-03-04,USA,Texas,21J (Delta),Delta
2535280,tiger/USA/FL-22-000348-001/2022,2022-01-02,2022-03-04,USA,Florida,21J (Delta),Delta


## Creating data sets as of given observation dates

In [7]:
def count_sequences_with_submission_date(metadata):    
    grouped = metadata.groupby(["date", "country", "variant"]) 
    
    def compute_delay(x):
        delays = (x["date_submitted"]
                  .value_counts(normalize=False)
                  .rename_axis('date_submitted')
                  .reset_index(name='sequences'))
        
        delays = delays.sort_values("date_submitted")
        return delays
    
    out = (grouped
           .apply(compute_delay)
           .reset_index()
           .drop(columns=["level_3"])
           .rename(columns={"country": "location"}))
    return out

delayed = count_sequences_with_submission_date(metadata)
delayed

,date,location,variant,date_submitted,sequences
0,2022-01-01,Australia,Delta,2022-01-04,1
1,2022-01-01,Australia,Delta,2022-01-06,1
2,2022-01-01,Australia,Delta,2022-01-11,1
3,2022-01-01,Australia,Delta,2022-01-12,2
4,2022-01-01,Australia,Delta,2022-01-13,10
...,...,...,...,...,...
66621,2022-06-30,United Kingdom,other,2022-07-07,13
66622,2022-06-30,United Kingdom,other,2022-07-08,1
66623,2022-06-30,United Kingdom,other,2022-07-11,1
66624,2022-06-30,United Kingdom,other,2022-07-20,1


In [8]:
def observe_sequence_counts(delayed, obs_date=None):
    # Given an observation date as well as counts of sequences and their submission dates,
    # Reconstruct data available on observation date
    
    obs_seq = delayed.copy()
    
    # Filter to sequences submitted on or before date
    if obs_date:
        obs_seq = obs_seq[obs_seq["date_submitted"] < obs_date]
        
    # Sum across remaining sequences
    obs_seq = (obs_seq
     .groupby(["date", "location", "variant"])["sequences"]
     .sum()).reset_index()
    
    # Sort data
    obs_seq = obs_seq.sort_values(["location", "variant", "date"])
        
    # Remove entries with no observed sequences
    obs_seq = obs_seq[obs_seq["sequences"] > 0]
    
    return obs_seq

In [9]:
# Testing observation dates
obs_seq = observe_sequence_counts(delayed, obs_date="2022-06-30")
obs_seq.date.max()

Timestamp('2022-06-26 00:00:00')

In [10]:
def make_observation_dates(obs_interval=7):
    total_interval = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days
    ndays = np.arange(90 + 14, total_interval, obs_interval).astype(int)
    if total_interval not in ndays:
        ndays = np.append(ndays, total_interval)

    obs_dates = [pd.to_datetime(start_date) + pd.Timedelta(nday, 'd') for nday in ndays]
    obs_dates = [date.strftime("%Y-%m-%d") for date in obs_dates]
    return obs_dates

In [11]:
obs_dates = make_observation_dates()
obs_dates

['2022-04-15',
 '2022-04-22',
 '2022-04-29',
 '2022-05-06',
 '2022-05-13',
 '2022-05-20',
 '2022-05-27',
 '2022-06-03',
 '2022-06-10',
 '2022-06-17',
 '2022-06-24',
 '2022-06-30']

In [12]:
# "https://covid.ourworldindata.org/data/owid-covid-data.csv"
raw_cases = pd.read_csv("https://covid.ourworldindata.org/data/owid-covid-data.csv")
raw_cases["location"].replace({"United States": "USA"}, inplace=True) #Map United States to USA
raw_cases = raw_cases[raw_cases.location.isin(locations)]
raw_cases = (raw_cases[["date", "location", "new_cases"]]
             .rename(columns={"new_cases": "cases"}))
raw_cases['cases'] = raw_cases['cases'].fillna(0)

In [14]:
import os

# Now we just loop over observation dates
for date in obs_dates:
    
    # Observe sequences up to this date
    obs_seq = observe_sequence_counts(delayed, obs_date=date)
    
    # Filter to most recent 104 days
    min_date = (pd.to_datetime(date) - pd.Timedelta(90 + 14,'d'))
    obs_seq = obs_seq[obs_seq.date > min_date]
    
    # Remove most recent 14 days due to bias
    max_date = (pd.to_datetime(date) - pd.Timedelta(14,'d'))
    obs_seq = obs_seq[obs_seq.date <= max_date]

    # Add cases up to present 
    obs_cases = raw_cases[raw_cases.date <= date]
    obs_cases = obs_cases[pd.to_datetime(obs_cases.date) > min_date]

    
    # Export file
    path = "../data/time_stamped/" + date
    if not os.path.exists(path):
        os.makedirs(path)
        
    # Make sure we have the folder
    obs_seq.to_csv(f"{path}/seq_counts_{date}.tsv", sep="\t", index = False)
    obs_cases.to_csv(f"{path}/case_counts_{date}.tsv", sep="\t", index = False)

In [15]:
# Get final truth sets
obs_seq = observe_sequence_counts(delayed, obs_date=None)
obs_cases = raw_cases

# Make sure we have the folder
path = "../data/time_stamped/truth"
if not os.path.exists(path):
        os.makedirs(path)
        
obs_seq.to_csv(f"{path}/seq_counts_truth.tsv", sep="\t", index = False)
obs_cases.to_csv(f"{path}/case_counts_truth.tsv", sep="\t", index = False)